# Defining MLP

In [268]:
import os
os.chdir('/Users/mikolajmroz/Developer/Computational_Intelligence_Methods')
print(os.getcwd())

/Users/mikolajmroz/Developer/Computational_Intelligence_Methods


In [269]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [270]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [271]:
def sigmoid_derivative(x):
    # Corrected sigmoid derivative to properly compute the derivative
    return sigmoid(x) * (1 - sigmoid(x))

In [272]:
def mse(predictions, targets):
    return np.mean((predictions - targets) ** 2)

In [273]:
class DataScaler:
    def __init__(self, method="standardization"):
        self.method = method
        self.min = None
        self.max = None
        self.mean = None
        self.std = None

    def fit_transform(self, data):
        if self.method == "min_max":
            return self.fit_transform_min_max(data)
        elif self.method == "standardization":
            return self.fit_transform_standardization(data)
        else:
            raise ValueError("Unsupported scaling method")

    def transform(self, data):
        if self.method == "min_max":
            return self.transform_min_max(data)
        elif self.method == "standardization":
            return self.transform_standardization(data)
        else:
            raise ValueError("Unsupported scaling method")

    def inverse_transform(self, data):
        if self.method == "min_max":
            return self.inverse_transform_min_max(data)
        elif self.method == "standardization":
            return self.inverse_transform_standardization(data)
        else:
            raise ValueError("Unsupported scaling method")

    def fit_transform_min_max(self, data):
        self.min = np.min(data, axis=0)
        self.max = np.max(data, axis=0)
        return (data - self.min) / (self.max - self.min)

    def transform_min_max(self, data):
        return (data - self.min) / (self.max - self.min)

    def inverse_transform_min_max(self, data):
        return data * (self.max - self.min) + self.min

    def fit_transform_standardization(self, data):
        self.mean = np.mean(data, axis=0)
        self.std = np.std(data, axis=0)
        return (data - self.mean) / self.std

    def transform_standardization(self, data):
        return (data - self.mean) / self.std

    def inverse_transform_standardization(self, data):
        return data * self.std + self.mean

In [274]:
class MLP:
    def __init__(self, sizes, activation_fn='sigmoid'):
        self.layer_sizes = sizes
        
        # Initialize weights with He initialization and biases with zeros
        self.layer_weights = [
            np.random.randn(next_layer, prev_layer) * np.sqrt(2.0 / (prev_layer + next_layer))
            for prev_layer, next_layer in zip(sizes[:-1], sizes[1:])
        ]
        self.layer_biases = [np.zeros((neurons, 1)) for neurons in sizes[1:]]

    def display_weights_biases(self):
        # Print the final weights and biases after training
        print("Final Weights and Biases:")
        for layer_index, (weights, biases) in enumerate(zip(self.layer_weights, self.layer_biases)):
            print(f"Layer {layer_index + 1} Weights:\n{weights}")
            print(f"Layer {layer_index + 1} Biases:\n{biases}")

    def propagate_forward(self, input_activation):
        # Forward pass through the network to get the output
        activations = [input_activation]  # Store all layer activations
        for biases, weights in zip(self.layer_biases[:-1], self.layer_weights[:-1]):
            input_activation = sigmoid(np.dot(weights, input_activation) + biases)
            activations.append(input_activation)
        # Apply linear activation for the last layer
        input_activation = np.dot(self.layer_weights[-1], input_activation) + self.layer_biases[-1]
        activations.append(input_activation)
        return activations[-1], activations

    def backward_propagation(self, input_val, true_val):
        # Backpropagate error and calculate gradient for weights and biases
        weight_gradients = [np.zeros(weight.shape) for weight in self.layer_weights]
        bias_gradients = [np.zeros(bias.shape) for bias in self.layer_biases]
        final_act, acts = self.propagate_forward(input_val)
        z_values = [np.dot(weight, act) + bias for weight, bias, act in zip(self.layer_weights, self.layer_biases, acts[:-1])]

        # Calculate error at the output layer
        error = self.cost_derivative(final_act, true_val)
        bias_gradients[-1] = error
        weight_gradients[-1] = np.dot(error, acts[-2].T)

        # Backpropagate the error to previous layers
        for l in range(2, len(self.layer_sizes)):
            sigmoid_grad = sigmoid_derivative(z_values[-l])
            error = np.dot(self.layer_weights[-l + 1].T, error) * sigmoid_grad
            bias_gradients[-l] = error
            weight_gradients[-l] = np.dot(error, acts[-l - 1].T)

        return weight_gradients, bias_gradients

    def update_batch(self, batch, learn_rate, regularization, total_size, optimization_method, beta, epsilon=None):
        gradient_w = [np.zeros(weight.shape) for weight in self.layer_weights]
        gradient_b = [np.zeros(bias.shape) for bias in self.layer_biases]
        for input_val, true_val in batch:
            delta_gradient_w, delta_gradient_b = self.backward_propagation(input_val, true_val)
            gradient_w = [w + dw for w, dw in zip(gradient_w, delta_gradient_w)]
            gradient_b = [b + db for b, db in zip(gradient_b, delta_gradient_b)]

        if optimization_method == 'momentum':
            if not hasattr(self, 'velocity_weights'):
                self.velocity_weights = [np.zeros_like(w) for w in self.layer_weights]
                self.velocity_biases = [np.zeros_like(b) for b in self.layer_biases]
                
            self.velocity_weights = [beta * vw + (1 - beta) * gw for vw, gw in zip(self.velocity_weights, gradient_w)]
            self.velocity_biases = [beta * vb + (1 - beta) * gb for vb, gb in zip(self.velocity_biases, gradient_b)]
            self.layer_weights = [(1 - learn_rate * (regularization / total_size)) * w - learn_rate * vw
                                  for w, vw in zip(self.layer_weights, self.velocity_weights)]
            self.layer_biases = [b - learn_rate * vb for b, vb in zip(self.layer_biases, self.velocity_biases)]
            
        elif optimization_method == 'rmsprop':
            if not hasattr(self, 'squared_gradients_weights'):
                self.squared_gradients_weights = [np.zeros_like(w) for w in self.layer_weights]
                self.squared_gradients_biases = [np.zeros_like(b) for b in self.layer_biases]
                
            self.squared_gradients_weights = [beta * sgw + (1 - beta) * (gw**2)
                                              for sgw, gw in zip(self.squared_gradients_weights, gradient_w)]
            self.squared_gradients_biases = [beta * sgb + (1 - beta) * (gb**2)
                                             for sgb, gb in zip(self.squared_gradients_biases, gradient_b)]
            self.layer_weights = [(1 - learn_rate * (regularization / total_size)) * w - 
                                  (learn_rate / (np.sqrt(sgw) + epsilon)) * gw
                                  for w, sgw, gw in zip(self.layer_weights, self.squared_gradients_weights, gradient_w)]
            self.layer_biases = [b - (learn_rate / (np.sqrt(sgb) + epsilon)) * gb
                                 for b, sgb, gb in zip(self.layer_biases, self.squared_gradients_biases, gradient_b)]
    
    def train(self, training_set, epoch_count, learn_rate, batch_size_input, regularization=0.0,
              visual_interval=100, optimization_method='momentum', beta=0.9, epsilon=1e-8, X_val_scaled = None, y_val_scaled = None, X_val = None, y_val = None,
             scaler_X = None, scaler_y = None):
        data_size = len(training_set)
        rate_init = learn_rate
        mse_history = []  # To store MSE values if tracking is enabled
    
        # Determine mini_batch_size based on input type
        if isinstance(batch_size_input, float):  # Input is percentage
            mini_batch_size = max(1, min(data_size, int(data_size * batch_size_input / 100)))
        elif isinstance(batch_size_input, int):  # Input is fixed value
            mini_batch_size = max(1, min(data_size, batch_size_input))
        else:
            raise ValueError("batch_size_input must be an integer (fixed size) or a float (percentage of dataset)")
        
        for epoch in range(epoch_count):
            np.random.shuffle(training_set)
            mini_batches = [training_set[k:k + mini_batch_size] for k in range(0, data_size, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_batch(mini_batch, learn_rate, regularization, data_size, optimization_method, beta, epsilon)
    
            learn_rate = rate_init / (1 + 0.01 * epoch)

            if epoch % visual_interval == 0 or epoch == epoch_count - 1:
                # Generate predictions on the scaled test data
                predictions_scaled_mse = np.array(
                    [self.propagate_forward(x.reshape(-1, 1))[0] for x in X_val_scaled])            
        
                predictions_mse = scaler_y.inverse_transform(predictions_scaled_mse.reshape(-1, 1))
        
                current_mse = mse(predictions_mse, y_val)
                mse_history.append(current_mse)
                print(f'Epoch {epoch}, MSE: {current_mse}')


    def cost_derivative(self, output_acts, true_val):
        # Calculate the derivative of the cost function
        return (output_acts - true_val)

    
    
    
    # def visualize_weights(self):
    #     # Visualize how the weights change over time during training
    #     epochs, layer_weights = zip(*self.weight_history)
    #     for layer_index, weights in enumerate(zip(*layer_weights)):
    #         plt.plot(epochs, [np.mean(np.abs(weight)) for weight in weights], label=f'Layer {layer_index + 1}')
        
    #     plt.xlabel('Epoch')
    #     plt.ylabel('Average Absolute Weight')
    #     plt.title('Weight Change Over Time')
    #     plt.legend()
    #     plt.show()


In [275]:
def plot_mse(mse_history):
    plt.plot(mse_history)
    plt.title('MSE Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Mean Squared Error')
    plt.show()

# Loading data

In [276]:
df_train_square_large = pd.read_csv('./data/regression/square-large-training.csv')
df_test_square_large = pd.read_csv('./data/regression/square-large-test.csv')

In [277]:
df_train_steps_large = pd.read_csv('./data/regression/steps-large-training.csv')
df_test_steps_large = pd.read_csv('./data/regression/steps-large-test.csv')

In [278]:
df_train_multimodal_large = pd.read_csv('./data/regression/multimodal-large-training.csv')
df_test_multimodal_large = pd.read_csv('./data/regression/multimodal-large-test.csv')

In [279]:
X_train_square_large = df_train_square_large['x'].values.reshape(-1, 1)
y_train_square_large = df_train_square_large['y'].values.reshape(-1, 1)

In [280]:
X_test_square_large = df_test_square_large['x'].values.reshape(-1, 1)
y_test_square_large = df_test_square_large['y'].values.reshape(-1, 1)

In [281]:
X_train_steps_large = df_train_steps_large['x'].values.reshape(-1, 1)
y_train_steps_large = df_train_steps_large['y'].values.reshape(-1, 1)

In [282]:
X_test_steps_large = df_test_steps_large['x'].values.reshape(-1, 1)
y_test_steps_large = df_test_steps_large['y'].values.reshape(-1, 1)

In [283]:
X_train_multimodal_large = df_train_multimodal_large['x'].values.reshape(-1, 1)
y_train_multimodal_large = df_train_multimodal_large['y'].values.reshape(-1, 1)

In [284]:
X_test_multimodal_large = df_test_multimodal_large['x'].values.reshape(-1, 1)
y_test_multimodal_large = df_test_multimodal_large['y'].values.reshape(-1, 1)

### square-large dataset

In [285]:
scaler_X = DataScaler("standardization")
scaler_y = DataScaler("standardization")

In [286]:
# Fit and transform the training data
X_train_scaled = scaler_X.fit_transform(X_train_square_large)
y_train_scaled = scaler_y.fit_transform(y_train_square_large)


In [287]:
# Fit and transform the testing data
X_test_scaled = scaler_X.fit_transform(X_test_square_large)
y_test_scaled = scaler_y.fit_transform(y_test_square_large)

In [288]:
training_data = [(x.reshape(-1, 1), y) for x, y in zip(X_train_square_large, y_train_square_large)]

In [289]:
mlp_square_1_5 = MLP([1, 6, 1] )

In [290]:
training_data_scaled = [
    (x.reshape(-1, 1), y) for x, y in zip(X_train_scaled, y_train_scaled)
]

In [291]:
mlp_square_1_5.train(
    training_data_scaled, epoch_count=1100, learn_rate=0.01, batch_size_input=10,
    optimization_method='rmsprop', beta=0.9, epsilon=1e-8, 
    X_val_scaled = X_test_scaled, y_val_scaled = y_test_scaled, X_val = X_test_square_large, y_val = y_test_square_large, scaler_y = scaler_y
)

Epoch 0, MSE: 11253.481054198925
Epoch 100, MSE: 2166.829014578336
Epoch 200, MSE: 2157.9310636178166
Epoch 300, MSE: 2147.4326198770145
Epoch 400, MSE: 2163.1072024901464
Epoch 500, MSE: 2161.41359914755
Epoch 600, MSE: 2188.940974329798
Epoch 700, MSE: 2174.438564359337
Epoch 800, MSE: 2161.388581841089
Epoch 900, MSE: 2197.766864393379
Epoch 1000, MSE: 2155.298645607983
Epoch 1099, MSE: 2192.74135519729


#### testing

In [292]:
plot_mse(mse_history)

NameError: name 'mse_history' is not defined

In [ ]:
mse_history

In [293]:
# Scale the test data using the transform method
X_test_scaled = scaler_X.transform(X_test_square_large)

In [294]:
# Generate predictions on the scaled test data
predictions_scaled = np.array(
    [mlp_square_1_5.propagate_forward(x.reshape(-1, 1))[0] for x in X_test_scaled]
)

In [295]:
# Correctly denormalize predictions
predictions = scaler_y.inverse_transform(predictions_scaled.reshape(-1, 1))

# Calculate MSE score
for i in range(len(predictions)):
    print("predicted value: ",  predictions[i], "actual value: ", y_test_square_large[i])
mse_score = mse(predictions, y_test_square_large)

print(f"MSE Score: {mse_score}")

predicted value:  [92.49634772] actual value:  [38.0481491]
predicted value:  [-124.15894983] actual value:  [-116.30927998]
predicted value:  [23.80967439] actual value:  [-22.98888333]
predicted value:  [227.66352981] actual value:  [163.49243459]
predicted value:  [-85.37598737] actual value:  [-50.79309017]
predicted value:  [79.24607534] actual value:  [26.14905307]
predicted value:  [-63.40510764] actual value:  [-95.99910634]
predicted value:  [48.00804044] actual value:  [-1.67979449]
predicted value:  [-40.43846984] actual value:  [11.84130953]
predicted value:  [8.20802677] actual value:  [-36.5786339]
predicted value:  [-7.20368902] actual value:  [-49.85187483]
predicted value:  [5.74652955] actual value:  [-38.70966897]
predicted value:  [-110.52366609] actual value:  [-89.65420844]
predicted value:  [69.21171334] actual value:  [151.89903669]
predicted value:  [-50.92060943] actual value:  [-86.1941224]
predicted value:  [85.48055414] actual value:  [172.6273849]
predicte

### multimodal-large dataset

In [454]:
# Initialize the scaler for X and y with the desired scaling method
scaler_X = DataScaler(method="min_max")
scaler_y = DataScaler(method="min_max")

In [455]:
mlp_multimodal = MLP([1, 10, 10, 1])

[array([[-0.12186476],
       [-1.07150803],
       [ 0.16918471],
       [-1.22704036],
       [ 0.29415834],
       [ 1.90776648],
       [ 0.36723362],
       [-2.43005663],
       [-0.64179758],
       [ 0.37628698]]), array([[-0.24349819, -0.09504987, -0.52955823,  0.02423583, -0.41818198,
        -0.24164405, -0.63978819, -0.19689873,  0.13058712, -0.13465842],
       [ 0.45080659, -0.02159449, -0.05549681,  0.04086744, -0.88827374,
        -0.09297092, -0.04961104,  0.28958064, -0.29969568, -0.09535983],
       [-0.0363863 ,  0.04247979,  0.27433519, -0.18349764, -0.10128933,
        -0.5755126 ,  0.33429007, -0.38199381,  0.3185607 , -0.34067407],
       [-0.17732828,  0.18565865,  0.40309407, -0.15555208, -0.05641572,
        -0.31815183,  0.3544479 , -0.21599497,  0.00092928,  0.16623052],
       [ 0.02547752,  0.19896638,  0.10654038, -0.22999851,  0.18712494,
        -0.16101919, -0.42042455, -0.12106941,  0.19802837,  0.25886289],
       [ 0.72251926,  0.00950327, -0.30287

In [456]:
# Fit and transform the training data
X_train_scaled3 = scaler_X.fit_transform(X_train_multimodal_large)
y_train_scaled3 = scaler_y.fit_transform(y_train_multimodal_large)


In [457]:
training_data_scaled3 = [
    (x.reshape(-1, 1), y) for x, y in zip(X_train_scaled3, y_train_scaled3)
]

In [458]:
mlp_multimodal.train(
    training_data_scaled3, epochs=500, learning_rate=1, batch_size=1
)

#### testing

In [459]:
# Scale the test data using the transform method
X_test_scaled3 = scaler_X.transform(X_test_multimodal_large)

In [460]:
# Generate predictions on the scaled test data
predictions_scaled3 = np.array(
    [mlp_multimodal.feedforward(x.reshape(-1, 1))[0] for x in X_test_scaled3]
)

In [461]:
# Correctly denormalize predictions
predictions3 = scaler_y.inverse_transform(predictions_scaled3.reshape(-1, 1))

# Calculate MSE score
for i in range(len(predictions3)):
    print("predicted value: ",  predictions3[i], "actual value: ", y_test_multimodal_large[i])
mse_score3 = mse(predictions3, y_test_multimodal_large)

print(f"MSE Score: {mse_score3}")

predicted value:  [-98.61762757] actual value:  [-98.2081664]
predicted value:  [-57.55949139] actual value:  [-55.28389066]
predicted value:  [96.81754344] actual value:  [100.29999676]
predicted value:  [-6.10210099] actual value:  [2.7206294]
predicted value:  [-76.14507474] actual value:  [-75.99163605]
predicted value:  [31.59826141] actual value:  [36.4875116]
predicted value:  [23.87751148] actual value:  [32.9071937]
predicted value:  [43.80285829] actual value:  [54.59378752]
predicted value:  [69.20004884] actual value:  [75.88595471]
predicted value:  [13.85803755] actual value:  [15.93020098]
predicted value:  [37.17074423] actual value:  [36.41048972]
predicted value:  [101.31296911] actual value:  [100.17958374]
predicted value:  [-96.29114011] actual value:  [-96.19000955]
predicted value:  [101.23356624] actual value:  [100.29859577]
predicted value:  [-66.26932765] actual value:  [-64.55778832]
predicted value:  [61.17476116] actual value:  [73.08482928]
predicted valu

In [388]:
# mse<40, zalozenie spelnione

### steps-large dataset

In [572]:
df_train_steps_large = pd.read_csv("data/regression/steps-large-test.csv")

In [573]:
scaler_X = DataScaler(method="standardization")
scaler_y = DataScaler(method="standardization")


In [574]:
df_test_steps_large = pd.read_csv("data/regression/steps-large-test.csv")

X_test_steps_small = df_test_steps_large["x"].values.reshape(-1, 1)
y_test_steps_small = df_test_steps_large["y"].values.reshape(-1, 1)

In [575]:
mlp_steps = MLP([1, 5, 5, 1])

[array([[ 0.22499592],
       [ 0.66393297],
       [ 0.81579735],
       [-0.93381363],
       [-1.56887485]]), array([[ 0.48408992, -0.90893581,  0.74451279,  0.26491836, -0.60762049],
       [ 0.21036234,  0.76803741,  0.00444413,  0.28424456, -0.40550138],
       [ 0.40071939, -0.39049337,  0.0533959 ,  0.28357303, -0.51281637],
       [-0.09733522,  0.20233363, -0.15237367, -0.91242117, -0.4788253 ],
       [ 0.66472232,  0.17079079,  0.40099151,  0.12026128,  0.05577142]]), array([[ 0.33198014, -0.44144792, -0.19640651, -0.21308176,  0.46809915]])]


In [576]:
# Fit and transform the training data
X_train_scaled2 = scaler_X.fit_transform(X_train_steps_small)
y_train_scaled2 = scaler_y.fit_transform(y_train_steps_small)


In [577]:
training_data_scaled2 = [
    (x.reshape(-1, 1), y) for x, y in zip(X_train_scaled2, y_train_scaled2)
]

In [578]:
mlp_steps.train(
    training_data_scaled2, epochs=5000, learning_rate=0.1, batch_size=10
)

#### testing

In [579]:
# Scale the test data using the transform method
X_test_scaled2 = scaler_X.transform(X_test_steps_small)

In [580]:
# Generate predictions on the scaled test data
predictions_scaled2 = np.array(
    [mlp_steps.feedforward(x.reshape(-1, 1))[0] for x in X_test_scaled2]
)

In [581]:
# Correctly denormalize predictions
predictions2 = scaler_y.inverse_transform(predictions_scaled2.reshape(-1, 1))

# Calculate MSE score
for i in range(len(predictions2)):
    print("predicted value: ",  predictions2[i], "actual value: ", y_test_steps_small[i])
mse_score2 = mse(predictions2, y_test_steps_small)

print(f"MSE Score: {mse_score2}")

predicted value:  [-75.68118152] actual value:  [-80]
predicted value:  [92.72419888] actual value:  [80]
predicted value:  [-88.27325684] actual value:  [-80]
predicted value:  [-70.89335599] actual value:  [-80]
predicted value:  [-78.9464397] actual value:  [-80]
predicted value:  [18.76146075] actual value:  [0]
predicted value:  [21.79168549] actual value:  [0]
predicted value:  [-87.52498457] actual value:  [-80]
predicted value:  [-74.5456806] actual value:  [-80]
predicted value:  [25.60761617] actual value:  [0]
predicted value:  [93.10415354] actual value:  [80]
predicted value:  [62.59932508] actual value:  [80]
predicted value:  [96.49685706] actual value:  [80]
predicted value:  [-75.53582894] actual value:  [-80]
predicted value:  [46.21620138] actual value:  [80]
predicted value:  [78.11865653] actual value:  [80]
predicted value:  [-18.89331856] actual value:  [0]
predicted value:  [146.8935984] actual value:  [160]
predicted value:  [-81.84228729] actual value:  [-80]


In [524]:
# nie ma optymalnego mse :((((